# `linearmodels`

In [1]:
import pandas as pd
from linearmodels.panel.data import PanelData
from linearmodels.panel import FirstDifferenceOLS
import wooldridge

## 説明

`linearmodels`は`statsmodels`を補完する目的として開発されている。主に，パネルデータ，操作変数法を使った推定法やGMMを扱う場合には非常に重宝するパッケージである。しかし，`linearmodels`は`statsmodels`の両方を使う上で以下の点に注意する必要がある。
* 推定結果などのメソッドや属性が共通化されているわけではない。次の表に３つの例を挙げる。


|              | 推定結果の<br> 表を表示 | 残差を<br> 表示する<br> メソッド | 標準誤差を<br> 取得する<br> 属性 |
|-------------:|:-----------------------:|:--------------------------------:|:--------------------------------:|
| statsmodels  | .summary()              | .resid                           | .bse                             |
| linearmodels | .summary()              | .resids                          | .std_errors                      |


* `statsmodels`も`linearmodels`も回帰式を文字列で指定できるが，定数項の指定する方法が異なる。
    * `statsmodels`では，定数項は自動的に追加される，定数項を省く場合は`-1`を追加する。
    * `linearmodels`では，定数項は自動的に追加されない。定数項を入れる場合は`1`を追加する。
* `fit()`メソッドの挙動も共通化されていない。
    * `linearmodels`の`fit()`に何のオプションも指定せずにOLS推定すると係数の推定量は同じだが，標準誤差や$t$値などが異なる。同じにするためには次のように２つのオプションを設定しなくてはならない。
    ```
    .fit(cov_type='unadjusted', debiased=True)
    ```
    * `cov_type`は不均一分散頑健共分散行列推定のオプション
        * デフォルトは`robust`（不均一分散頑健的共分散行列推定）で`statsmodels`の`HC1`と等しい。
    * `debiased`は共分散行列推定の自由度のオプション（小標本の場合の調整）
        * デフォルトは`False`

**（注意）**

以下では`.fit()`のオプションは指定せず，デフォルトのまま議論を続ける。

---
以下では`linearmodels`を使うが，そのためには`DataFrame`を`MultiIndex`に変換する必要がある。以下では，まず`MultiIndex`について説明し，その後に`linearmodel`にある`PanelData`関数について説明する

まず，このトピックで使用するパッケージを導入する。

## `Pandas`の`MultiIndex`

### 説明

パネル・データを扱うために必要な`Pandas`の`MultiIndex`について説明する。`MultiIndex`とは行や列のラベルが階層的になった`DataFrame`や`Series`を指す。以下では，`DataFrame`の行における`MultiIndex`を説明する。

まずデータを読み込む。

In [2]:
# url の設定
url = 'https://raw.githubusercontent.com/Haruyama-KobeU/Haruyama-KobeU.github.io/master/data/data4.csv'

# 読み込み
df = pd.read_csv(url)
df

,year,country,gdp,inv,con,pop
0,2000,Australia,90,30.0,50.0,10
1,2001,Australia,100,40.0,80.0,11
2,2002,Australia,120,NaN,70.0,16
3,2000,Japan,100,20.0,80.0,8
4,2001,Japan,95,25.0,70.0,9
5,2002,Japan,93,21.0,72.0,10
6,2000,UK,100,30.0,70.0,11
7,2001,UK,110,39.0,71.0,12
8,2002,UK,115,55.0,NaN,14


行・列ともにラベルの階層は１つずつとなっている。`set_index()`を使い行に`MultiIndex`を作成するが，引数に

$$\left[\text{第０id},\text{第１id}\right]$$

とし階層インデックス化する。ここでパネル・データ分析をする上で以下のルールに従うことにする。
* 第０id：観察単位（例えば，消費者，企業，国）
* 第１id：時間（例えば，年，四半期）

次の例では`country`と`year`の行をそれぞれ第０インデックス，第１インデックスに指定する。

In [3]:
df = df.set_index(['country', 'year'])#.sort_index()
df

gdp   inv   con  pop
country   year                      
Australia 2000   90  30.0  50.0   10
          2001  100  40.0  80.0   11
          2002  120   NaN  70.0   16
Japan     2000  100  20.0  80.0    8
          2001   95  25.0  70.0    9
          2002   93  21.0  72.0   10
UK        2000  100  30.0  70.0   11
          2001  110  39.0  71.0   12
          2002  115  55.0   NaN   14

階層インデックスが綺麗に並んでいるが，元のデータの並び方によっては階層インデックスが期待通りに並ばない場合がありえる。その場合は，メソッド`sort_index()`を使うと良いだろう。

---
`MultiIndex`を解除するにはメソッド`.reset_index()`を使う。

In [4]:
df.reset_index()

,country,year,gdp,inv,con,pop
0,Australia,2000,90,30.0,50.0,10
1,Australia,2001,100,40.0,80.0,11
2,Australia,2002,120,NaN,70.0,16
3,Japan,2000,100,20.0,80.0,8
4,Japan,2001,95,25.0,70.0,9
5,Japan,2002,93,21.0,72.0,10
6,UK,2000,100,30.0,70.0,11
7,UK,2001,110,39.0,71.0,12
8,UK,2002,115,55.0,NaN,14


### 要素，行，列の抽出

`MultiIndex`のまま要素・列・行の抽出およぼスライシングには様々な方法があり，複雑である。特に，スライシングをしたい場合，一番簡単なのは`reset_index()`で通常の`DataFrame`に戻し，スライシングし新たな`DataFrame`を作成するだけでも十分であろう。

以下では，`.loc[]`を使い`MultiIndex`のままでの抽出方法について簡単に説明する。その際，以下のルールは変わらない。

$$.\text{loc}\left[\text{行の指定},\text{列の指定}\right]$$

ただ，`行の指定`にリストやタプルを使うことになる（`列の指定`も同じ）。

他の方法については[このサイト](https://note.nkmk.me/python-pandas-multiindex-indexing/)と[このサイト](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html)が参考になる。

#### １つの観察単位の抽出

１つの要素を抽出する場合は，タプルを使う。例えば，日本の2001年の`gdp`を抽出したい場合。

In [5]:
df.loc[('Japan',2001), 'gdp']

95.0

#### 行の抽出

上の例で列の指定を`:`にすると，指定した行に対して全ての列を抽出できる。

In [6]:
df.loc[('Japan',2001), :]

gdp    95.0
inv    25.0
con    70.0
pop     9.0
Name: (Japan, 2001), dtype: float64

この場合，列に対してスライシングも可能。

In [7]:
df.loc[('Japan',2001), 'gdp':'con']

gdp    95.0
inv    25.0
con    70.0
Name: (Japan, 2001), dtype: float64

指定した行に対して個別に複数列を抽出したい場合は，タプルを使う。

In [8]:
df.loc[('Japan',2001), ('gdp','con')]

gdp    95.0
con    70.0
Name: (Japan, 2001), dtype: float64

複数行の抽出にはリストで指定する。

In [9]:
df.loc[(['Japan','UK'],[2001,2002]), :]

gdp   inv   con  pop
country year                      
Japan   2001   95  25.0  70.0    9
        2002   93  21.0  72.0   10
UK      2001  110  39.0  71.0   12
        2002  115  55.0   NaN   14

#### 第０インデックスの観察単位の全て

第０インデックスにある，ある観察単位の全てのデータだけを抽出したい場合は，通常の`Pandas`の場合と同じ。

In [10]:
df.loc['Japan', :]

,gdp,inv,con,pop
year,,,,
2000,100,20.0,80.0,8
2001,95,25.0,70.0,9
2002,93,21.0,72.0,10


複数の場合。

In [11]:
df.loc[['Japan','India'], :]

gdp   inv   con  pop
country year                      
Japan   2000  100  20.0  80.0    8
        2001   95  25.0  70.0    9
        2002   93  21.0  72.0   10

#### 列の抽出

通常の`Pandas`と同じ。`Series`を返す場合。

In [12]:
df.loc[:,'gdp']

country    year
Australia  2000     90
           2001    100
           2002    120
Japan      2000    100
           2001     95
           2002     93
UK         2000    100
           2001    110
           2002    115
Name: gdp, dtype: int64

`[]`を使うと，`DataFrame`として抽出できる。

In [13]:
df.loc[:,['gdp']]

gdp
country   year     
Australia 2000   90
          2001  100
          2002  120
Japan     2000  100
          2001   95
          2002   93
UK        2000  100
          2001  110
          2002  115

複数列抽出の場合。

In [14]:
df.loc[:,['gdp','inv']]

gdp   inv
country   year           
Australia 2000   90  30.0
          2001  100  40.0
          2002  120   NaN
Japan     2000  100  20.0
          2001   95  25.0
          2002   93  21.0
UK        2000  100  30.0
          2001  110  39.0
          2002  115  55.0

スライシングも使える。

In [15]:
df.loc[:,'gdp':'con']

gdp   inv   con
country   year                 
Australia 2000   90  30.0  50.0
          2001  100  40.0  80.0
          2002  120   NaN  70.0
Japan     2000  100  20.0  80.0
          2001   95  25.0  70.0
          2002   93  21.0  72.0
UK        2000  100  30.0  70.0
          2001  110  39.0  71.0
          2002  115  55.0   NaN

#### 第１インデックスのある年だけの抽出

一番簡単な方法は`reset_index()`を使い今まで習った関数を使う。

In [16]:
df.reset_index().query('year == 2000')

,country,year,gdp,inv,con,pop
0,Australia,2000,90,30.0,50.0,10
3,Japan,2000,100,20.0,80.0,8
6,UK,2000,100,30.0,70.0,11


複数年の場合。

In [17]:
df.reset_index().query('year in [2000,2002]')

,country,year,gdp,inv,con,pop
0,Australia,2000,90,30.0,50.0,10
2,Australia,2002,120,NaN,70.0,16
3,Japan,2000,100,20.0,80.0,8
5,Japan,2002,93,21.0,72.0,10
6,UK,2000,100,30.0,70.0,11
8,UK,2002,115,55.0,NaN,14


上と同じ結果。

In [18]:
df.reset_index().query('year not in [2001]')

,country,year,gdp,inv,con,pop
0,Australia,2000,90,30.0,50.0,10
2,Australia,2002,120,NaN,70.0,16
3,Japan,2000,100,20.0,80.0,8
5,Japan,2002,93,21.0,72.0,10
6,UK,2000,100,30.0,70.0,11
8,UK,2002,115,55.0,NaN,14


## `linearmodels`の`PanelData`

`linearmodels`では`MultiIndex`化された`DataFrame`をそのまま読み込み推定することができる。一方で，`linearmodels`の関数`PanelData`を使い`MultiIndex`化された`DataFrame`を`PanelData`オブジェクトに変換すると分析に必要な計算を簡単にできるようになる。必須ではないが，知っていて損はしない関数である。

まず`df`を`PanelData`オブジェクトに変換する。

In [19]:
dfp = PanelData(df)
dfp

PanelData
                  gdp   inv   con   pop
country   year                         
Australia 2000   90.0  30.0  50.0  10.0
          2001  100.0  40.0  80.0  11.0
          2002  120.0   NaN  70.0  16.0
Japan     2000  100.0  20.0  80.0   8.0
          2001   95.0  25.0  70.0   9.0
          2002   93.0  21.0  72.0  10.0
UK        2000  100.0  30.0  70.0  11.0
          2001  110.0  39.0  71.0  12.0
          2002  115.0  55.0   NaN  14.0
PanelData object, id: 0x7f9e9485f0a0

---
属性`shape`は，`PanelData`の変数の数を表示する。以下が返り値の内容である。

$$
\left(\text{変数の数},\text{期間数},\text{観察単位の数}\right)
$$

In [20]:
dfp.shape

(4, 3, 3)

* 変数の数：4（列にある変数）
* 期間数：3（年）
* 観察単位の数：3（国）

---
データセットには欠損値がある場合がある。観察単位数が$N$で期間数が$T$の場合，観測値の数は$n=N\times T$となるが，次の2つを区別する。
* balanced panel data：$n=N\times T$（観察単位に対して全ての期間の全ての変数に欠損値がない）
* unbalanced panel data：$n<N\times T$（欠損値がある）

balanced か unbalancedかは以下のコードで確認できる。まず，メソッド`count()`を使う。

In [21]:
dfp.count()

,gdp,inv,con,pop
country,,,,
Australia,3,2,3,3
Japan,3,3,3,3
UK,3,3,2,3


観察単位（国）に対して，それぞれの変数に欠損値ではない観測値がいくつ存在するかを`DataFrame`として返す。期間数は3なので，3より低い数字があれば欠損値の存在を表す。例えば，Australiaの`inv`には欠損値がある。

次のコードは，欠損値がある場合には`True`を返す。ここで`nobs`は期間数（この場合3）を返す属性である。

In [22]:
dfp.count() == dfp.nobs

,gdp,inv,con,pop
country,,,,
Australia,True,False,True,True
Japan,True,True,True,True
UK,True,True,False,True


`all()`は，列に対して全ての要素が`True`の場合のみ`True`を返すので，これを使い確認できる。

In [23]:
(dfp.count() == dfp.nobs).all()

gdp     True
inv    False
con    False
pop     True
dtype: bool

`( )`はその中を先に評価する，という意味（数学と同じ）。変数が多い場合，`all()`を2回使うと全ての変数に対して評価するので便利である。

In [24]:
(dfp.count() == dfp.nobs).all().all()

False

`False`なので unbalanced panel data ということが確認できた。

---
変数の観察単位毎の平均の計算

In [25]:
dfp.mean()

,gdp,inv,con,pop
country,,,,
Australia,103.333333,35.000000,66.666667,12.333333
Japan,96.000000,22.000000,74.000000,9.000000
UK,108.333333,41.333333,70.500000,12.333333


---
変数の時間毎の平均の計算

In [26]:
dfp.mean('time')

,gdp,inv,con,pop
year,,,,
2000,96.666667,26.666667,66.666667,9.666667
2001,101.666667,34.666667,73.666667,10.666667
2002,109.333333,38.000000,71.000000,13.333333


---
変数の平均からの乖離　$x-\bar{x}$，$\bar{x}$は平均。

In [27]:
dfp.demean()

PanelData
                      gdp        inv        con       pop
country   year                                           
Australia 2000 -13.333333  -5.000000 -16.666667 -2.333333
          2001  -3.333333   5.000000  13.333333 -1.333333
          2002  16.666667        NaN   3.333333  3.666667
Japan     2000   4.000000  -2.000000   6.000000 -1.000000
          2001  -1.000000   3.000000  -4.000000  0.000000
          2002  -3.000000  -1.000000  -2.000000  1.000000
UK        2000  -8.333333 -11.333333  -0.500000 -1.333333
          2001   1.666667  -2.333333   0.500000 -0.333333
          2002   6.666667  13.666667        NaN  1.666667
PanelData object, id: 0x7f9e9483c4f0

---
変数の１階階差の計算　$x_t-x_{t-1}$

In [28]:
dfp.first_difference()

PanelData
                 gdp   inv   con  pop
country   year                       
Australia 2001  10.0  10.0  30.0  1.0
Japan     2001  -5.0   5.0 -10.0  1.0
          2002  -2.0  -4.0   2.0  1.0
UK        2001  10.0   9.0   1.0  1.0
PanelData object, id: 0x7f9e923ccd30

---
（注意）

`DataFrame`のメソッドは`PanelData`オブジェクトには使えない。

従って，`DataFrame`のメソッド（例えば，行や列の抽出）を使う場合，`DataFrame`に変換する必要がある。その際，`PanelData`オブジェクトの属性`.dataframe`を使うことができる。

In [29]:
dfp.dataframe.loc['Japan',:]

,gdp,inv,con,pop
year,,,,
2000,100.0,20.0,80.0,8.0
2001,95.0,25.0,70.0,9.0
2002,93.0,21.0,72.0,10.0


## １階差分推定（再考）

ここでは`linearmodels`を使い，以前行った１階差分推定を再考する。データ`crime4`を使う。

In [30]:
crime4 = wooldridge.data('crime4')
crime4.head()

,county,year,crmrte,prbarr,prbconv,prbpris,avgsen,polpc,density,taxpc,...,lpctymle,lpctmin,clcrmrte,clprbarr,clprbcon,clprbpri,clavgsen,clpolpc,cltaxpc,clmix
0,1,81,0.039885,0.289696,0.402062,0.472222,5.61,0.001787,2.307159,25.697630,...,-2.433870,3.006608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,82,0.038345,0.338111,0.433005,0.506993,5.59,0.001767,2.330254,24.874252,...,-2.449038,3.006608,-0.039376,0.154542,0.074143,0.071048,-0.003571,-0.011364,-0.032565,0.030857
2,1,83,0.030305,0.330449,0.525703,0.479705,5.80,0.001836,2.341801,26.451443,...,-2.464036,3.006608,-0.235316,-0.022922,0.193987,-0.055326,0.036879,0.038413,0.061477,-0.244732
3,1,84,0.034726,0.362525,0.604706,0.520104,6.89,0.001886,2.346420,26.842348,...,-2.478925,3.006608,0.136180,0.092641,0.140006,0.080857,0.172213,0.026930,0.014670,-0.027331
4,1,85,0.036573,0.325395,0.578723,0.497059,6.55,0.001924,2.364896,28.140337,...,-2.497306,3.006608,0.051825,-0.108054,-0.043918,-0.045320,-0.050606,0.020199,0.047223,0.172125


`county`と`year`を使い`MultiIndex`化する。

In [31]:
crime4 = crime4.set_index(['county','year'])
crime4.head()

crmrte    prbarr   prbconv   prbpris  avgsen     polpc  \
county year                                                             
1      81    0.039885  0.289696  0.402062  0.472222    5.61  0.001787   
       82    0.038345  0.338111  0.433005  0.506993    5.59  0.001767   
       83    0.030305  0.330449  0.525703  0.479705    5.80  0.001836   
       84    0.034726  0.362525  0.604706  0.520104    6.89  0.001886   
       85    0.036573  0.325395  0.578723  0.497059    6.55  0.001924   

              density      taxpc  west  central  ...  lpctymle   lpctmin  \
county year                                      ...                       
1      81    2.307159  25.697630     0        1  ... -2.433870  3.006608   
       82    2.330254  24.874252     0        1  ... -2.449038  3.006608   
       83    2.341801  26.451443     0        1  ... -2.464036  3.006608   
       84    2.346420  26.842348     0        1  ... -2.478925  3.006608   
       85    2.364896  28.140337     0        1  ... -2.497306  3.006608   

             clcrmrte  clprbarr  clprbcon  clprbpri  clavgsen   clpolpc  \
county year                                                               
1      81         NaN       NaN       NaN       NaN       NaN       NaN   
       82   -0.039376  0.154542  0.074143  0.071048 -0.003571 -0.011364   
       83   -0.235316 -0.022922  0.193987 -0.055326  0.036879  0.038413   
       84    0.136180  0.092641  0.140006  0.080857  0.172213  0.026930   
       85    0.051825 -0.108054 -0.043918 -0.045320 -0.050606  0.020199   

              cltaxpc     clmix  
county year                      
1      81         NaN       NaN  
       82   -0.032565  0.030857  
       83    0.061477 -0.244732  
       84    0.014670 -0.027331  
       85    0.047223  0.172125  

[5 rows x 57 columns]

次に`PanelData`オブジェクトに変換しデータの特徴を調べる。

In [32]:
crime4p = PanelData(crime4)
crime4p.shape

(57, 7, 90)

* 57: 変数の数
* 7: 期間数（年）
* 90：観察単位の数（人数）

次に，balanced もしくは unbalanced data set かを確認する。

In [33]:
(crime4p.count()==crime4p.nobs).all().all()

False

---
実際に回帰式を書くことにする。使い方は`statsmodels`と似ている。
* `FirstDifferenceOLS`モジュールの関数`.from_formula`を使い次のように引数を指定する。

$$\text{.from_formula}(\text{回帰式}, \text{データ})$$

* 定数項を入れることはできない仕様となっている。
* ここでは，以前の推定結果と比べるために，ダミー変数`d82`を追加する。

In [34]:
formula = 'lcrmrte ~ d82 + d83 + d84 + d85 + d86 + d87 + lprbarr + \
                lprbconv + lprbpris + lavgsen + lpolpc'

* １階差分モデルのインスタンスの作成

In [35]:
mod_dif = FirstDifferenceOLS.from_formula(formula, data=crime4)

* `statsmodels`と同じように，そこから得た結果にメソッド`.fit()`を使い計算し結果が返される。

In [36]:
res_dif = mod_dif.fit()

＜結果の表示方法＞
1. `res_dif`を実行。
1. `res_dif`に関数`print()`を使うと見やすい。
1. `res_dif`には属性`summary`が用意されているが，表示方法1と同じ。
1. `summary`には属性`tables`があり，２つの表がリストとして格納されている。
    * `tables[0]`：検定統計量の表（`print()`を使うと見やすくなる）
    * `tables[1]`：係数の推定値やp値などの表（`print()`を使うと見やすくなる）

In [37]:
print(res_dif.summary.tables[1])

                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
d82            0.0077     0.0171     0.4522     0.6513     -0.0258      0.0412
d83           -0.0844     0.0235    -3.5998     0.0003     -0.1305     -0.0384
d84           -0.1247     0.0287    -4.3366     0.0000     -0.1811     -0.0682
d85           -0.1216     0.0331    -3.6670     0.0003     -0.1867     -0.0564
d86           -0.0863     0.0367    -2.3539     0.0189     -0.1584     -0.0143
d87           -0.0378     0.0400    -0.9455     0.3448     -0.1163      0.0407
lprbarr       -0.3275     0.0300    -10.924     0.0000     -0.3864     -0.2686
lprbconv      -0.2381     0.0182    -13.058     0.0000     -0.2739     -0.2023
lprbpris      -0.1650     0.0260    -6.3555     0.0000     -0.2161     -0.1140
lavgsen       -0.0218     0.0221    -0.9850     0.32

推定結果は以前のものと同じである。